# step1_build_MSM
This Jupyter notebook coverts featurized trajectories in `insulin-dimer/step0_featurize/outputs` into discrete state trajectories (`state_arr`)

Following are the tasks this notebook does:
- Concatenating featurized trajectories `cvs_df`:
I first ran 2.5 ns of 18816 (28x28x24) trajectories, and I extended them to 5.0 ns. 
This leads to some CVs being computed separately for first and last 2.5 ns, while others were computed as a whole (5ns)
- Definition of dimer and separte monomer state: `dim_arr`, ($A$), `mon_arr` ($B$)
- Select and scale CVs to convert them to features for defining Markov states based on $k$-mean clustering `proc_df`
- Define Markov states `state_arr` by running $k$-means on `proc_df` space


## "Open only if necessary. Code can be highly confusing."

In [1]:
import dill
import numpy as np
import pandas as pd
from time import time

# 1. cvs_cat_df
## cvs_0_df
workdir=f"/project/dinner/kjeong/insulin/pipeline"
cvs_0_arr = np.load(f"{workdir}/step2_procfeat/CV_data.npy")
cvs_0_label = np.load(f"{workdir}/step2_procfeat/CV_label.npy")
cvs_0_df=pd.DataFrame({key: value for key, value in zip(cvs_0_label, cvs_0_arr.T)})

Nframe = cvs_0_arr.shape[0]
ntraj = 28*28*24
length = 1000

## cvs_1_df
input_path=f"{workdir}/step1_feat/output/extend_5ns/"

with open(f"{input_path}/HeavyContact.pkl", 'rb') as f:
    HeavyContact_ls = dill.load(f)
HeavyContact = np.hstack([tmp.timeseries for tmp in HeavyContact_ls])

with open(f"{input_path}/IRMSD.pkl", 'rb') as f:
    IRMSD_ls = dill.load(f)
IRMSD = np.hstack([tmp.rmsd[:, 2] for tmp in IRMSD_ls])

with open(f"{input_path}/ISolv.pkl", 'rb') as f:
    ISolv_ls = dill.load(f)
NSharedWater = np.hstack([tmp.timeseries[:, 0, 1] for tmp in ISolv_ls])
NWater = np.hstack([tmp.timeseries[:, 0, 0] for tmp in ISolv_ls])

with open(f"{input_path}/angle_open.pkl", 'rb') as f:
    angle_open_ls = dill.load(f)
Angle_Open = np.vstack([(180/np.pi) * tmp[1].timeseries for tmp in angle_open_ls]) # (9408000, 2)

with open(f"{input_path}/angle.pkl", 'rb') as f:
    angle_ls = dill.load(f)
Angle = np.vstack([(180/np.pi) * tmp[0].timeseries for tmp in angle_ls]) # (9408000, 4) "phi-alpha", "phi-alpha(me)", "phi-beta", "phi-beta(me)",

with open(f"{input_path}/Euler.pkl", 'rb') as f:
    Euler_ls = dill.load(f)
Euler = np.vstack([tmp.timeseries for tmp in Euler_ls]) # (9408000, 6) "r_com, "theta", "phi", "Theta", "Phi", "Psi""
Euler[:, 0] = Euler[:, 0] / 10
Euler[:, 1:] = Euler[:, 1:] * 180/np.pi

with open(f"{input_path}/BBagchi.pkl", 'rb') as f:
    BBagchi_ls = dill.load(f)
R_COM_Bagchi = []
N_Bagchi = []
for r_com_tmp, n_tmp in BBagchi_ls:
    R_COM_Bagchi.append(r_com_tmp.timeseries)
    N_Bagchi.append(n_tmp.timeseries)
R_COM_Bagchi = np.hstack(R_COM_Bagchi)/10
N_Bagchi = np.hstack(N_Bagchi)
#'R_com', 'CrossContact(7A)'

cvs_1_df = pd.DataFrame(
    {
        'phi-alpha': Angle[:, 0],
        'phi-alpha(me)': Angle[:, 1],
        'phi-beta': Angle[:, 2],
        'phi-beta(me)': Angle[:, 3],
        "phi-open(ch1)": Angle_Open[:, 0],
        "phi-open(ch2)": Angle_Open[:, 1],
        'r_com': Euler[:, 0],
        'theta': Euler[:, 1],
        'phi': Euler[:, 2],
        'Theta': Euler[:, 3],
        'Phi': Euler[:, 4],
        'Psi': Euler[:, 5],
        'Nwater(Interface)': NWater,
        'Shared Nwater(Interface)': NSharedWater,
        'R_com': R_COM_Bagchi,
        'CrossContact(7A)': N_Bagchi
    }
)

In [4]:
# 2. raw_feat_cat_df
## a. Update cvs_0,1_df for `beta`, `alpha`, `zip-Dist``
## b. Use to make #4. `feat_cat_arr` to define `state_arr`
with open(f"{workdir}/step1_feat/output/distance.pkl", 'rb') as f:
    ref_feat = dill.load(f)
feat_0_arr=np.concatenate([dt.timeseries for dt in ref_feat])/10 #A to nm

with open(f"{input_path}/distance.pkl", 'rb') as f:
    raw_feat = dill.load(f)
feat_1_arr=np.concatenate([dt.timeseries for dt in raw_feat])/10 #A to nm

beta = np.sum(feat_1_arr[:, [6, 7, 8], [8, 7, 6]], axis=1)/3
alpha = np.sum(feat_1_arr[:, [0, 0, 1, 2, 2, 3, 3], [2, 3, 3, 2, 0, 0, 1]], axis=1)/7

###a2. zip, open
cvs_0_df['zip-Dist'] = np.sum(feat_0_arr[:, [-1, -2], [4, 3]] - feat_0_arr[:, [4, 3], [-1, -2]], axis=1)
cvs_1_df['zip-Dist'] = np.sum(feat_1_arr[:, [-1, -2], [4, 3]] - feat_1_arr[:, [4, 3], [-1, -2]], axis=1)

cvs_0_df['phi-open'] = cvs_0_df['phi-open(ch1)'] + cvs_0_df['phi-open(ch2)']
cvs_1_df['phi-open'] = cvs_1_df['phi-open(ch1)'] + cvs_1_df['phi-open(ch2)']

### cvs_df Concateante
cvs_dic = {}
for key in cvs_1_df.keys():
    cvs_dic[key] = np.ravel(np.hstack((
        cvs_0_df.loc[:, key].to_numpy().reshape(ntraj, int(length/2)),
        cvs_1_df.loc[:, key].to_numpy().reshape(ntraj, int(length/2))
    )))
cvs_df = pd.DataFrame(cvs_dic)
print(cvs_df.keys())

Index(['phi-alpha', 'phi-alpha(me)', 'phi-beta', 'phi-beta(me)',
       'phi-open(ch1)', 'phi-open(ch2)', 'r_com', 'theta', 'phi', 'Theta',
       'Phi', 'Psi', 'Nwater(Interface)', 'Shared Nwater(Interface)', 'R_com',
       'CrossContact(7A)', 'zip-Dist', 'phi-open'],
      dtype='object')


In [ ]:
# 3. dim_arr, mon_arr, other_arr
# dim_0_arr, mon_0_arr, other_0_arr
tpt_arr = np.load(f"{workdir}/step3_DGA/tpt_data.npy")
tpt_label= np.load(f"{workdir}/step3_DGA/tpt_label.npy")
tpt_0_df=pd.DataFrame({key: value for key, value in zip(tpt_label, tpt_arr.T)})

dim_0_arr = tpt_0_df.state.to_numpy().astype(int)==0
mon_0_arr = tpt_0_df.state.to_numpy().astype(int)==199
other_0_arr = ~(dim_0_arr | mon_0_arr)

# dim_1_arr, mon_1_arr, other_1_arr
dim_1_arr = (IRMSD<2) & (Angle[:, 0]>120) & (Angle[:, 0]<135) & (beta<0.55) & (alpha<0.8)
mon_1_arr = (IRMSD>10) & (HeavyContact==0) & (NSharedWater == 0)
other_1_arr = ~(dim_1_arr | mon_1_arr)

# Concatenate
dim_arr = np.ravel(np.hstack([dim_0_arr.reshape(28*28*24, 500), dim_1_arr.reshape(28*28*24, 500)]))
mon_arr = np.ravel(np.hstack([mon_0_arr.reshape(28*28*24, 500), mon_1_arr.reshape(28*28*24, 500)]))
other_arr = np.ravel(np.hstack([other_0_arr.reshape(28*28*24, 500), other_1_arr.reshape(28*28*24, 500)]))
#np.save("/project/dinner/kjeong/insulin/pipeline/step7_5ns/step1_cvs_state/AB_arr/dim_arr.npy", dim_arr)
#np.save("/project/dinner/kjeong/insulin/pipeline/step7_5ns/step1_cvs_state/AB_arr/mon_arr.npy", mon_arr)
#np.save("/project/dinner/kjeong/insulin/pipeline/step7_5ns/step1_cvs_state/AB_arr/other_arr.npy", other_arr)

In [5]:
feat_arr = np.concatenate((
    feat_0_arr.reshape((ntraj, int(length/2), 10, 10)),
    feat_1_arr.reshape((ntraj, int(length/2), 10, 10)),), axis=1)
print(feat_arr.shape)
feat_arr = feat_arr.reshape(int(ntraj*length), 10, 10)
print(feat_arr.shape)

(18816, 1000, 10, 10)
(18816000, 10, 10)


In [6]:
beta_id = ([6, 7, 8], [8, 7, 6]) #3
alpha_id = ([0, 0, 1, 2, 2, 3, 3], [2, 3, 3, 2, 0, 0, 1]) #7
gamma1_id = ([1, 1, 3, 3, 4, 4], [8, 9, 8, 9, 8, 9]) #6
gamma2_id = ([8, 9, 8, 9, 8, 9], [1, 1, 3, 3, 4, 4]) #6

cvs_df["beta"] = np.sum(feat_arr[:, beta_id[0], beta_id[1]], axis=1)/3
cvs_df["alpha"] = np.sum(feat_arr[:, alpha_id[0], alpha_id[1]], axis=1)/7
cvs_df["gamma1"] = np.sum(feat_arr[:, gamma1_id[0], gamma1_id[1]], axis=1)/6
cvs_df["gamma2"] = np.sum(feat_arr[:, gamma2_id[0], gamma2_id[1]], axis=1)/6

In [7]:
contact_ids = np.hstack((beta_id, alpha_id, gamma1_id, gamma2_id))
nat_mean = np.mean(feat_arr[dim_arr][:, contact_ids[0, :10], contact_ids[1, :10]], axis=0)
nat_dist = feat_arr[:, contact_ids[0, :10], contact_ids[1, :10]] - nat_mean
nat_mask = nat_dist < 0
nat_contact = np.exp(-nat_dist**2/(2*(0.6**2)))
nat_contact[nat_mask] = 1

nonnat_mean = np.mean(nat_mean)
nonnat_dist = feat_arr[:, contact_ids[0, 10:], contact_ids[1, 10:]] - nonnat_mean
nonnat_mask = nonnat_dist < 0
nonnat_contact = np.exp(-nonnat_dist**2/(2*(0.6**2)))
nonnat_contact[nonnat_mask] = 1

print(nat_contact.shape, nonnat_contact.shape)

cvs_df['betac'] = np.sum(nat_contact[:, :3], axis=1)/3
cvs_df['alphac'] = np.sum(nat_contact[:, 3:], axis=1)/7
cvs_df['gamma1c'] = np.sum(nonnat_contact[:, :6], axis=1)/6
cvs_df['gamma2c'] = np.sum(nonnat_contact[:, 6:], axis=1)/6

(18816000, 10) (18816000, 12)


In [9]:
# 1. Zipping
input_path_whole=f"{workdir}/step1_feat/output/whole_5ns/"
with open(f"{input_path_whole}/ZIP.pkl", 'rb') as f:
    ZIP_ls = dill.load(f)
Zip = (180/np.pi)*np.vstack([zip_ls.timeseries for zip_ls in ZIP_ls]).T #(4, nframe) # 4 angles (PROB, 21), (PROB, 8 29), (PROD, 21), (PROD, 8 29)
cvs_df["zip-turn"] = Zip[0]+(np.pi-Zip[2])
cvs_df["zip-term"] = Zip[1]+(np.pi-Zip[3])
#cvs_df["zip-turn(D)"] = Zip[2]+(np.pi-Zip[0])
#cvs_df["zip-term(D)"] = Zip[3]+(np.pi-Zip[])

# 2. All Contact
with open(f"{input_path_whole}/Allcontact_native.pkl", 'rb') as f:
    allcontact_dic = dill.load(f)
Contact = np.vstack([allcontact for allcontact in allcontact_dic.values()]) #(4, nframe) (All, native, semi-native, non-native)
for arr, key in zip(Contact, ["All", "native", "semi-native", "non-native"]):
    cvs_df[key] = arr

with open(f"{input_path_whole}/Detach.pkl", 'rb') as f:
    detach_ls = dill.load(f)
DETACH = np.vstack([tmp.timeseries for tmp in detach_ls]) * 180 / np.pi
cvs_df["detach1"] = DETACH[:, 0]
cvs_df["detach2"] = DETACH[:, 1]

with open(f"{input_path_whole}/BBagchiQ.pkl", 'rb') as f:
    bbagchiQ_ls = dill.load(f)
BBagchiQ = np.hstack([tmp.timeseries for tmp in bbagchiQ_ls])
cvs_df["Q_Bagchi"] = BBagchiQ

# 3. Extra hassle with hydrogen bond on 24-26', 26-24'

type_hb = [(0, '24', '24'), (0, '24', '26'), (0, '26', '24'), (0, '26', '26'),\
          (1, '24', '24'), (1, '24', '26'), (1, '26', '24'), (1, '26', '26'),]
ntype_hb=len(type_hb)
type_hb_label= []
for o_tmp, ch1_tmp, ch2_tmp in type_hb:
    BondorBridge = "Hbridge" if o_tmp else "Hbond"
    type_hb_label.append(f"{BondorBridge}( {ch1_tmp}-{ch2_tmp}' )")


hbb0= np.load(f"{workdir}/step1_feat/output/hbb_arr.npy")
hbb1= np.load(f"{workdir}/step1_feat/output/extend_5ns/hbb_arr.npy")
nathb_cnt_arr_ls = []
for hbb in [hbb0, hbb1]:
    nathb_arr = np.zeros((ntype_hb, hbb.shape[0]), dtype=bool)
    for i0, (o_tmp, ch1_tmp, ch2_tmp) in enumerate(type_hb):
        bool_o_tmp= (hbb[:, 1] == o_tmp)
        bool_ch_tmp = np.zeros((2, len(bool_o_tmp)),dtype=bool)
        for i1, (i2, ch_str) in enumerate(zip([2, 3], [ch1_tmp, ch2_tmp])):
            if ch_str == '24':
                bool_ch_tmp[i1] = (hbb[:, i2] < 2)
            elif ch_str == '26':
                bool_ch_tmp[i1] = (hbb[:, i2]> 3)
            else:
                raise KeyError
        nathb_arr[i0]=bool_o_tmp & bool_ch_tmp[0] & bool_ch_tmp[1]
    nathb_cnt_arr = np.zeros((ntype_hb, Nframe), dtype=np.int8)
    for i0 in range(ntype_hb):
        uq_tmp, cnt_tmp=np.unique(hbb[np.nonzero(nathb_arr[i0])[0], 0], return_counts=True)
        nathb_cnt_arr[i0, uq_tmp] = cnt_tmp
    nathb_cnt_arr_ls.append(nathb_cnt_arr)

nathb_cnt_arr = np.concatenate(
    (nathb_cnt_arr_ls[0].reshape(-1, ntraj, int(length/2)),
    nathb_cnt_arr_ls[1].reshape(-1, ntraj, int(length/2))), axis=2)
nathb_cnt_arr = nathb_cnt_arr.reshape(-1, ntraj*length).T

for i0, key in enumerate(type_hb_label):
    cvs_df[key] = nathb_cnt_arr[:, i0]

In [12]:
#cvs_df.to_pickle("/project/dinner/kjeong/insulin/pipeline/step7_5ns/step1_cvs_state/cvs_df.pkl")
print(cvs_df.shape)

In [7]:
CA_label = np.array([9, 12, 13, 16, 21, 23, 24, 25, 26, 29])
mychoice_feat_resid = np.array(
    [
        [ 9,  9], [ 9, 13], [13,  9], [13, 13],#Alpha-Alpha
        [24, 24], [24, 26], [26, 24], [26, 26],#Beta-Beta
        [21, 29], [24, 29], [26, 21],#Turn-Cterm
        [29, 21], [29, 24], [21, 26],#Cterm-Turn
        [21,  9], [26,  9], [29,  9], [21, 16], [26, 16], [29, 16],
        [ 9, 21], [ 9, 26], [ 9, 29], [16, 21], [16, 26], [16, 29]
    ]
)
def res2id(elem):
    return CA_label.tolist().index(elem)
mychoice_feat = np.array(list(map(res2id, np.ravel(mychoice_feat_resid)))).reshape(mychoice_feat_resid.shape)

def tanh_std(arr):
    mu = np.mean(arr, axis=0)
    sigma = np.std(arr, axis=0)
    return np.tanh((arr-mu)/(2*sigma))

def angle_std(arr):
    mu_angle= np.mean(arr[dim_arr], axis=0)
    return np.hstack((np.cos(np.deg2rad(arr-mu_angle)), np.sin(np.deg2rad(arr-mu_angle))))

# Processed distance
dist_0_arr=feat_0_arr[:, mychoice_feat[:, 0], mychoice_feat[:, 1]]
dist_1_arr=feat_1_arr[:, mychoice_feat[:, 0], mychoice_feat[:, 1]]
dist_arr = np.concatenate((dist_0_arr.reshape(ntraj, 500, 26), dist_1_arr.reshape(ntraj, 500, 26)), axis=1).reshape((2*Nframe, 26))
proc_dist = tanh_std(dist_arr)

# Processed angle
angle_of_interest =["phi-beta(me)", "phi-alpha(me)", 
                    "phi-open(ch1)", "phi-open(ch2)",
                    "theta", "phi", "Theta", "Phi", "Psi"]
proc_angle = angle_std(cvs_df.loc[:, angle_of_interest])
proc_rcom = cvs_df.loc[:, 'r_com'].to_numpy().reshape(2*Nframe, 1)/2

proc_arr = np.hstack((proc_rcom, proc_dist, proc_angle))

In [20]:
proc_lab = np.array(
    ["r_com"]+[f"B{i:02d}-B'{j:02d}" for i, j in mychoice_feat_resid] \
    +[f"cos({angle})" for angle in angle_of_interest]\
    +[f"sin({angle})" for angle in angle_of_interest]
)

In [22]:
proc_df = pd.DataFrame({lab: arr for lab, arr in zip(proc_lab, proc_arr.T)})
#proc_df.to_pickle(f"{workdir}/step7_5ns/step1_cvs_state/proc_df.pkl")
print(proc_df.shape)

In [5]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.neighbors import NearestNeighbors
from time import time

In [6]:
# Slicing
sliced_proc_arr = proc_arr.reshape((ntraj, length, 45))[:, 50:100, :].reshape((ntraj*50, 45))
sliced_other_arr = np.ravel(other_arr.reshape((ntraj, length))[:, 50:100])

# Number of Markov states `k_ls` and random state `rn_ls`
#k_ls = [100, 200, 400, 600]
k_ls = [1000, 1500, 2000, 3000]
rn_ls = [0, 1, 2]
neighbors_arr = np.zeros((len(k_ls), len(rn_ls)), dtype=object)
for i0, k in enumerate(k_ls):
    t0 = time()
    for i1, rn in enumerate(rn_ls):
        km = MiniBatchKMeans(n_clusters = k-2, random_state=rn).fit(sliced_proc_arr[sliced_other_arr])
        neighbors_arr[i0, i1] = NearestNeighbors(n_neighbors=1).fit(km.cluster_centers_)
    print(f"Kmeans with {k} clusters is done ({time()-t0:.2f} sec)")


Kmeans with 1000 clusters is done (14.92 sec)
Kmeans with 1500 clusters is done (18.80 sec)
Kmeans with 2000 clusters is done (22.39 sec)
Kmeans with 3000 clusters is done (34.80 sec)


In [7]:
# state_arr
state_arr = np.zeros((len(k_ls), len(rn_ls), 2*Nframe), dtype=int)
for i0, k in enumerate(k_ls):
    t0 = time()
    for i1, rn in enumerate(rn_ls):
        state_other = np.squeeze(neighbors_arr[i0, i1].kneighbors(proc_arr[other_arr], return_distance=False))
        state_arr[i0, i1, mon_arr] = k-1
        state_arr[i0, i1, other_arr] = state_other+1
    print(f"{k} clusters is done ({time()-t0:.2f} sec)")

1000 clusters is done (408.54 sec)
1500 clusters is done (615.60 sec)
2000 clusters is done (812.85 sec)
3000 clusters is done (1225.99 sec)


In [8]:
#np.save("/project/dinner/kjeong/insulin/pipeline/step7_5ns/step1_cvs_state/state_arr.npy", state_arr)
#np.save("/project/dinner/kjeong/insulin/pipeline/step7_5ns/step1_cvs_state/state_arr_hk.npy", state_arr)